In [10]:
import rasterio
from rasterio.mask import mask
import fiona
import os
import numpy as np
import pandas as pd

In [16]:
def clip_raster_by_multiple_polygons(raster_path, geojson_path, output_dir):
    """
    Обрезает растровое изображение для каждого полигона в GeoJSON и сохраняет 
    каждый результат в отдельный файл.
    """
    # Убедитесь, что выходная директория существует
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")

    # 1. Загрузка всех геометрий из GeoJSON
    with fiona.open(geojson_path, "r") as shapefile:
        features = list(shapefile)
    
    print(f"Найдено полигонов для обрезки: {len(features)}")

    # 2. Открытие исходного растрового изображения (один раз)
    with rasterio.open(raster_path) as src:
        
        # 3. Итерация по каждому полигону (Feature) отдельно
        for i, feature in enumerate(features):
            geometry = [feature["geometry"]] # Оборачиваем геометрию в список для функции mask()
            
            # --- Шаг обрезки ---
            # Применяем маску только для текущего полигона
            try:
                out_image, out_transform = mask(src, geometry, crop=True, nodata=0)
            except ValueError as e:
                print(f"Пропущена обрезка полигона {i}: {e}")
                continue # Пропускаем полигоны, которые не пересекают растр
                
            # --- Подготовка метаданных для нового файла ---
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1], # Правильные размеры
                "width": out_image.shape[2],  # Правильные размеры
                "transform": out_transform,
                "nodata": 0 # Устанавливаем черный/прозрачный фон для областей вне полигона
            })

            # --- Сохранение файла ---
            output_filename = f"clipped_segment_{i:03d}.tif"
            output_path = os.path.join(output_dir, output_filename)
            
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            print(f"  -> Сохранен файл: {output_path}")


def calculate_ndvi(red_band, nir_band):
    """
    Рассчитывает NDVI из массивов красного и ближнего инфракрасного каналов.
    """
    red_band = red_band.astype(float)
    nir_band = nir_band.astype(float)
    
    denominator = nir_band + red_band
    # Избегаем деления на ноль
    denominator[denominator == 0] = 1 

    ndvi = (nir_band - red_band) / denominator
    return ndvi

def load_and_process_ndvi_from_separate_files(rgb_path, ocn_path):
    """
    Загружает красный канал из RGB файла и NIR канал из OCN файла, 
    рассчитывает NDVI и сохраняет результат.

    :param rgb_path: Путь к файлу с каналами RGB.
    :param ocn_path: Путь к файлу с каналами OCN.
    :param output_path: Путь для сохранения карты NDVI.
    """
    
    # Предполагаемые индексы каналов (начинаются с 1):
    # RGB: 1=R, 2=G, 3=B. Мы берем КРАСНЫЙ канал (индекс 1).
    RED_BAND_INDEX = 1 
    # OCN: 1=O, 2=C, 3=Nir. Мы берем NIR канал (индекс 3).
    NIR_BAND_INDEX = 3 

    # 1. Загрузка Красного канала
    with rasterio.open(rgb_path) as src_rgb:
        red_band = src_rgb.read(RED_BAND_INDEX)
        # Метаданные берем из файла RGB, т.к. у него та же привязка, что и у OCN файла
        meta = src_rgb.meta.copy()

    # 2. Загрузка NIR канала
    with rasterio.open(ocn_path) as src_ocn:
        nir_band = src_ocn.read(NIR_BAND_INDEX)
    
    # Проверка на совпадение размеров изображений  
    if red_band.shape != nir_band.shape:  
        print(f"Размеры не совпадают: RGB={red_band.shape}, OCN={nir_band.shape}. Обрезаем...")
        
        # Определяем минимальные размеры
        min_height = min(red_band.shape[0], nir_band.shape[0])
        min_width = min(red_band.shape[1], nir_band.shape[1])
        
        # Обрезаем оба массива до общего размера
        red_band = red_band[:min_height, :min_width]
        nir_band = nir_band[:min_height, :min_width]

    # 3. Расчет NDVI
    ndvi_data = calculate_ndvi(red_band, nir_band)

    return np.nanmean(ndvi_data[ndvi_data != 0]) # Возвращаем среднее значение


In [7]:
# Сегментируем rgb изображение
clip_raster_by_multiple_polygons(
    raster_path="C:/Users/smolk/data/Field 2024/2024_07_03 Пшеница Выход в трубку - Колошение/rgb.tif",
    geojson_path="2024_выход_в_трубку-кущение_разметка.geojson",
    output_dir="output_segments_2024_выход_в_трубку-кущение" # Папка, куда будут сохранены все файлы
)

Найдено полигонов для обрезки: 148
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_000.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_001.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_002.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_003.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_004.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_005.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_006.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_007.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_008.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_009.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение\clipped_segment_010.t

In [8]:
# Сегментируем ocn изображение
clip_raster_by_multiple_polygons(
    raster_path="C:/Users/smolk/data/Field 2024/2024_07_03 Пшеница Выход в трубку - Колошение/ocn.tif",
    geojson_path="2024_выход_в_трубку-кущение_разметка.geojson",
    output_dir="output_segments_2024_выход_в_трубку-кущение_NIR" # Папка, куда будут сохранены все файлы
)

Найдено полигонов для обрезки: 148
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_000.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_001.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_002.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_003.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_004.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_005.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_006.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_007.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_008.tif
  -> Сохранен файл: output_segments_2024_выход_в_трубку-кущение_NIR\clipped_segment_009.tif
  -> Сохранен файл: output_segments_2024_выхо

In [9]:
# --- Пример использования для одной пары файлов ---

# Вам нужно будет указать пути к вашим обрезанным файлам
mean_ndvi_value = load_and_process_ndvi_from_separate_files(
    rgb_path="output_segments_2024_выход_в_трубку-кущение/clipped_segment_000.tif",
    ocn_path="output_segments_2024_выход_в_трубку-кущение_NIR/clipped_segment_000.tif"
)

print(f"Среднее NDVI для делянки 000: {mean_ndvi_value:.4f}")

Среднее NDVI для делянки 000: 0.2784


In [61]:
import pandas as pd
import os

# Создаем датафрейм для результатов
df_ndvi = pd.DataFrame(columns=['plot_id', 'mean_ndvi'])

# Предполагаем, что файлы называются типа "segment_001.tif", "segment_002.tif" и т.д.
# И что есть парные RGB и NIR файлы с одинаковыми номерами

for i in range(0, 148):  # Предположим, что делянки от 1 до 147
    plot_id = f"{i:03d}"  # Форматируем номер как 001, 002, ...
    
    rgb_path = f"output_segments_2024_выход_в_трубку-кущение/clipped_segment_{plot_id}.tif"
    ocn_path = f"output_segments_2024_выход_в_трубку-кущение_NIR/clipped_segment_{plot_id}.tif"
    
    # Проверяем, что оба файла существуют
    if os.path.exists(rgb_path) and os.path.exists(ocn_path):
        try:
            # Используем вашу готовую функцию
            mean_ndvi = load_and_process_ndvi_from_separate_files(
                rgb_path=rgb_path,
                ocn_path=ocn_path
            )
            
            # Добавляем в датафрейм
            df_ndvi.loc[len(df_ndvi)] = [plot_id, mean_ndvi]
            print(f"Делянка {plot_id}: NDVI = {mean_ndvi:.4f}")
            
        except Exception as e:
            print(f"Ошибка при обработке делянки {plot_id}: {e}")
    else:
        print(f"Файлы для делянки {plot_id} не найдены")

Делянка 000: NDVI = 0.2784
Делянка 001: NDVI = 0.2927
Делянка 002: NDVI = 0.3316
Делянка 003: NDVI = 0.3283
Размеры не совпадают: RGB=(636, 1932), OCN=(636, 1933). Обрезаем...
Делянка 004: NDVI = 0.3138
Делянка 005: NDVI = 0.2846
Делянка 006: NDVI = 0.3150
Делянка 007: NDVI = 0.3071
Делянка 008: NDVI = 0.3096
Делянка 009: NDVI = 0.2733
Делянка 010: NDVI = 0.2971
Делянка 011: NDVI = 0.3176
Делянка 012: NDVI = 0.2985
Делянка 013: NDVI = 0.3024
Делянка 014: NDVI = 0.3233
Делянка 015: NDVI = 0.3125
Делянка 016: NDVI = 0.3122
Делянка 017: NDVI = 0.3340
Делянка 018: NDVI = 0.3292
Делянка 019: NDVI = 0.3146
Делянка 020: NDVI = 0.2954
Делянка 021: NDVI = 0.2768
Делянка 022: NDVI = 0.3077
Делянка 023: NDVI = 0.3092
Делянка 024: NDVI = 0.3422
Делянка 025: NDVI = 0.3252
Делянка 026: NDVI = 0.3069
Делянка 027: NDVI = 0.3039
Делянка 028: NDVI = 0.3193
Делянка 029: NDVI = 0.3400
Делянка 030: NDVI = 0.3462
Делянка 031: NDVI = 0.2891
Делянка 032: NDVI = 0.2945
Делянка 033: NDVI = 0.3269
Делянка 034: N

In [62]:
df_ndvi['genotype'] = None

In [63]:
df_ndvi = df_ndvi.astype({'plot_id': np.int16})

In [64]:
df_ndvi

,plot_id,mean_ndvi,genotype
0,0,0.278411,None
1,1,0.292651,None
2,2,0.331634,None
3,3,0.328296,None
4,4,0.313839,None
...,...,...,...
143,143,0.276801,None
144,144,0.271906,None
145,145,0.263052,None
146,146,0.282051,None


In [55]:
for i in range(37):
    df_ndvi.iloc[i, 2] = i+1

In [59]:
df_ndvi.to_excel("df_ndvi.xlsx", index=False)